In [188]:
global t
t = 0

TERRAINS = {
    "W": 1,  
    "G": 1.2,
    "T": min(1.5, 0.1 * t),
    "M": 1.6,
}

In [189]:
def heuristic(a, b):
    """Manhattan distance multiplied by the lowest terrain cost (water)."""
    (x1, y1), (x2, y2) = a, b
    return (abs(x1 - x2) + abs(y1 - y2)) * TERRAINS["W"]

# First Grid:

In [190]:
grid = [
    list("WGGTG"),
    list("GMWMM"),
    list("GTMWG"),
    list("GGGMM"),
    list("TMMGW"),
    list("GWGMG"),
    list("GGMTM"),
    list("WGGWM"),
]

ROWS, COLS = len(grid), len(grid[0])
start = (0, 0)  
goal = (6, 4)   

In [191]:
def neighbors(node):
    x, y = node
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < ROWS and 0 <= ny < COLS:
            if TERRAINS[grid[nx][ny]] is not None:
                yield (nx, ny)

In [192]:
import heapq


def a_star(start, goal):
    global t
    open_heap = []
    heapq.heappush(open_heap, (0, start))
    came_from = {}
    g_score = {start: 0}
    opened_order, closed_order = [], []

    while open_heap:
        current_f, current = heapq.heappop(open_heap)
        closed_order.append(current)
        t += 1

        if current == goal:
            break

        for n in neighbors(current):
            tentative_g = g_score[current] + TERRAINS[grid[n[0]][n[1]]]
            if n not in g_score or tentative_g < g_score[n]:
                came_from[n] = current
                g_score[n] = tentative_g
                f = tentative_g + heuristic(n, goal)
                heapq.heappush(open_heap, (f, n))
                if n not in opened_order:
                    opened_order.append(n)

    # Reconstruct path
    path = []
    node = goal
    while node in came_from or node == start:
        path.append(node)
        if node == start:
            break
        node = came_from[node]
    path.reverse()
    return path, opened_order, closed_order

In [193]:
path, opened, closed = a_star(start, goal)

In [194]:
print("\nGrid:")
for r in range(ROWS):
    row_repr = ""
    for c in range(COLS):
        symbol = grid[r][c]
        if (r, c) == start:
            symbol = "S"
        elif (r, c) == goal:
            symbol = "G"
        elif (r, c) in path:
            symbol = "*"
        row_repr += symbol + " "
    print(row_repr)

print("\nOpened nodes (order added):")
print(opened)

print("\nClosed nodes (order processed):")
print(closed)

print("\nFinal path from S to G:")
print(path)



Grid:
S G G T G 
* M W M M 
* T M W G 
* G G M M 
* M M G W 
* * * * G 
G G M * G 
W G G W M 

Opened nodes (order added):
[(1, 0), (0, 1), (1, 1), (0, 2), (2, 0), (1, 2), (0, 3), (1, 3), (0, 4), (1, 4), (2, 3), (3, 3), (2, 2), (2, 4), (3, 4), (3, 0), (2, 1), (3, 1), (4, 1), (3, 2), (4, 2), (5, 1), (4, 0), (6, 1), (5, 0), (5, 2), (6, 2), (5, 3), (7, 1), (6, 0), (4, 3), (7, 0), (6, 3), (5, 4), (7, 3), (6, 4)]

Closed nodes (order processed):
[(0, 0), (0, 1), (1, 0), (0, 2), (0, 3), (0, 4), (1, 3), (2, 3), (1, 4), (2, 4), (1, 2), (2, 0), (2, 1), (3, 1), (3, 2), (2, 2), (4, 1), (5, 1), (5, 2), (6, 1), (3, 3), (4, 2), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (6, 0), (6, 1), (3, 3), (5, 3), (6, 3), (6, 4)]

Final path from S to G:
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (5, 3), (6, 3), (6, 4)]


In [195]:
EMOJIS = {
    'W': '💧',   
    'G': '🌾',   
    'M': '🪨',   
    'T': '🚗',   
}


def render(opened, closed, highlight=None):
    clear_output(wait=True)
    symbols = {tuple(start): 'S', tuple(goal): 'G'}
    for (r, c) in opened:   symbols[(r, c)] = '🔵'      
    for (r, c) in closed:   symbols[(r, c)] = '🟥'      
    if highlight: symbols[highlight] = '▶'

    w, _ = shutil.get_terminal_size(fallback=(80, 24))
    print('-' * w)
    for r in range(len(grid)):
        row = []
        for c in range(len(grid[0])):
            cell = symbols.get((r, c), grid[r][c])
            row.append(EMOJIS.get(cell, cell))         # letter to emoji
        print(' '.join(row))
    print('-' * w)

    print("Opened:", sorted(opened))
    print("Closed:", sorted(closed))


for cur, opened, closed in a_star_stream(start, goal):
    os.system('cls' if os.name == 'nt' else 'clear')
    render(opened, closed, highlight=cur)
    time.sleep(0.05)               


path = a_star_stream(start, goal)   

--------------------------------------------------------------------------------
🟥 🟥 🟥 🟥 🟥
🟥 🔵 🟥 🟥 🟥
🟥 🟥 🟥 🟥 🟥
🟥 🟥 🟥 🟥 🔵
🟥 🟥 🟥 🔵 💧
🟥 🟥 🟥 🟥 🔵
🟥 🟥 🔵 🟥 ▶
🔵 🔵 🌾 🔵 🪨
--------------------------------------------------------------------------------
Opened: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (7, 0), (7, 1), (7, 3)]
Closed: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2), (5, 3), (6, 0), (6, 1), (6, 3), (6, 4)]


# Second Grid

In [196]:
grid = [
    list("WGGTG"),
    list("GMWGG"),
    list("GTMWG"),
    list("GGGGW"),
    list("TMMGW"),
    list("GWGWG"),
    list("GGMTG"),
    list("WGGGG"),
]

ROWS, COLS = len(grid), len(grid[0])
start = (0, 0)  
goal = (6, 4)   

In [197]:
path, opened, closed = a_star(start, goal)

In [198]:
print("\nGrid:")
for r in range(ROWS):
    row_repr = ""
    for c in range(COLS):
        symbol = grid[r][c]
        if (r, c) == start:
            symbol = "S"
        elif (r, c) == goal:
            symbol = "G"
        elif (r, c) in path:
            symbol = "*"
        row_repr += symbol + " "
    print(row_repr)

print("\nOpened nodes (order added):")
print(opened)

print("\nClosed nodes (order processed):")
print(closed)

print("\nFinal path from S to G:")
print(path)


Grid:
S * * * G 
G M W * G 
G T M * G 
G G G * W 
T M M * W 
G W G * G 
G G M * G 
W G G G G 

Opened nodes (order added):
[(1, 0), (0, 1), (1, 1), (0, 2), (2, 0), (1, 2), (0, 3), (1, 3), (0, 4), (1, 4), (2, 3), (3, 3), (2, 2), (2, 4), (3, 4), (4, 3), (3, 2), (4, 4), (5, 4), (5, 3), (4, 2), (6, 3), (5, 2), (7, 3), (6, 2), (6, 4)]

Closed nodes (order processed):
[(0, 0), (0, 1), (1, 0), (0, 2), (0, 3), (0, 4), (1, 3), (2, 3), (1, 4), (2, 4), (3, 3), (3, 4), (4, 4), (4, 3), (5, 3), (6, 3), (6, 4)]

Final path from S to G:
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (6, 4)]


In [199]:
for cur, opened, closed in a_star_stream(start, goal):
    os.system('cls' if os.name == 'nt' else 'clear')
    render(opened, closed, highlight=cur)
    time.sleep(0.05)               


path = a_star_stream(start, goal) 

--------------------------------------------------------------------------------
🟥 🟥 🟥 🟥 🟥
🟥 🔵 🔵 🟥 🟥
🔵 🚗 🔵 🟥 🟥
🌾 🌾 🔵 🟥 🟥
🚗 🪨 🔵 🟥 🟥
🌾 💧 🔵 🟥 🔵
🌾 🌾 🔵 🟥 ▶
💧 🌾 🌾 🔵 🌾
--------------------------------------------------------------------------------
Opened: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 2), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4), (4, 2), (4, 3), (4, 4), (5, 2), (5, 3), (5, 4), (6, 2), (6, 3), (6, 4), (7, 3)]
Closed: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 3), (1, 4), (2, 3), (2, 4), (3, 3), (3, 4), (4, 3), (4, 4), (5, 3), (6, 3), (6, 4)]
